In [1]:

# 本次演示单轮对话机器人的训练
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
# 代码中使用， 不使用DataCollatorForLanguageModeling的原因是：生成的label是错误的（label是右移一位）
from transformers import DataCollatorForSeq2Seq, DataCollatorForLanguageModeling



In [2]:


ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds



Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:

ds[:3]


{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [4]:

tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh", cache_dir="bloom-389m-zh")
tokenizer



BloomTokenizerFast(name_or_path='Langboat/bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:

# 定义单轮对话的数据预处理

def process_func(example):
    max_length = 256
    input_ids, attention_mask, labels = [], [], []
    # 指令instruction中包含了用户的角色、用户指令、用户输入以及下一步系统助手的角色
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"] + "\n\nAssistant: "]).strip())
    response = tokenizer(example["output"] + tokenizer.eos_token)

    # 生成输入的id数据
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 截断
    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}



In [6]:

tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds



Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
from IPython.display import display
# 通过解码器验证数据集处理是否正确

# # 将input_ids重新编码回对应的文本输入
display(tokenizer.decode(tokenized_ds[0]["input_ids"]))

# # 过滤不需要计算loss的label，重新编码回对应的输出文本
display(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[0]["labels"]))))


'Human: 保持健康的三个提示。\n\n\nAssistant:以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

'以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [8]:

# 创建模型

model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh", cache_dir="bloom-389m-zh")


/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:

# 创建训练参数

args = TrainingArguments(output_dir="./bloom-trained",per_device_train_batch_size=16,
                         gradient_accumulation_steps=4, 
                         logging_steps=10, num_train_epochs=2)


In [10]:
trainer = Trainer(model=model,args=args,train_dataset=tokenized_ds, data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer))

In [11]:
trainer.train()

Step,Training Loss
10,3.054900
20,2.977700
30,2.965800
40,2.926700
50,2.826000
60,2.749400
70,2.746100
80,2.673300
90,2.618900
100,2.671800


TrainOutput(global_step=838, training_loss=2.0867599667114405, metrics={'train_runtime': 764.3769, 'train_samples_per_second': 70.274, 'train_steps_per_second': 1.096, 'total_flos': 1.4681551454208e+16, 'train_loss': 2.0867599667114405, 'epoch': 1.9964264443120905})

In [13]:


# 模型推理


from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device=0)



In [21]:
ipt = "Human: {} \n {}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "

pipe(ipt, do_sample=True, max_length=1)

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧通常是指在考试中的某种方法、技巧或者是步骤。有效的考试技巧能够帮助考试者更好地理解考试内容，提高考试效率。常见的考试技巧包括代入法、选择题、简答、阅读理解、分析题、论述题和综合分析等。除此之外，优秀的考试技巧还需要结合考生的不同知识和经验来制定，以保证其考生的感受和分析能够准确地传达信息。'}]

In [18]:
pipe??

Signature:      pipe(text_inputs, **kwargs)
Type:           TextGenerationPipeline
String form:    <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fdc18677580>
File:           ~/miniconda3/lib/python3.10/site-packages/transformers/pipelines/text_generation.py
Source:        
@add_end_docstrings(build_pipeline_init_args(has_tokenizer=True))
class TextGenerationPipeline(Pipeline):
    """
    Language generation pipeline using any `ModelWithLMHead`. This pipeline predicts the words that will follow a
    specified text prompt. When the underlying model is a conversational model, it can also accept one or more chats,
    in which case the pipeline will operate in chat mode and will continue the chat(s) by adding its response(s).
    Each chat takes the form of a list of dicts, where each dict contains "role" and "content" keys.

    Examples:

    ```python
    >>> from transformers import pipeline

    >>> generator = pipeline(model="openai-community/gpt2")
  